In [3]:
import random
from pathlib import Path
from collections import OrderedDict
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
import foolbox
from foolbox import PyTorchModel
from foolbox.attacks import (
    LinfBasicIterativeAttack,
    FGSM,
    PGD,
    L2DeepFoolAttack,
    L2CarliniWagnerAttack,
)
from torchvision import transforms
from PIL import Image
import torch.nn as nn


def _load_checkpoint_into_model(model, checkpoint_path, map_location="cpu", strict=True, verbose=False):
    ckpt = torch.load(checkpoint_path, map_location=map_location)

    if isinstance(ckpt, dict):
        for candidate in ("model_state", "model_state_dict", "state_dict", "state"):
            if candidate in ckpt:
                state = ckpt[candidate]
                break
        else:
            if all(isinstance(v, (torch.Tensor, type(None))) or hasattr(v, "shape") for v in ckpt.values()):
                state = ckpt
            else:
                nested = None
                for v in ckpt.values():
                    if isinstance(v, dict):
                        if nested is None or len(v) > len(nested):
                            nested = v
                state = nested if nested is not None else ckpt
    else:
        state = ckpt

    if not isinstance(state, dict):
        raise ValueError(f"Checkpoint {checkpoint_path} does not contain a state-dict (found type: {type(state)})")

    keys = list(state.keys())
    prefix = None
    for p in ("module.", "model."):
        cnt = sum(1 for k in keys if k.startswith(p))
        if cnt >= max(1, len(keys) // 2):
            prefix = p
            break

    if prefix:
        new_state = OrderedDict((k[len(prefix):], v) for k, v in state.items())
    else:
        new_state = state

    try:
        model.load_state_dict(new_state, strict=strict)
        if verbose:
            print(f"Loaded checkpoint {checkpoint_path} (strict={strict}).")
        return model
    except Exception as e:
        if verbose:
            print(f"Strict load failed: {e}. Trying non-strict load ...")
        res = model.load_state_dict(new_state, strict=False)
        if verbose:
            print("Loaded with strict=False. Missing keys:", getattr(res, "missing_keys", None))
            print("Unexpected keys:", getattr(res, "unexpected_keys", None))
        return model


def run_attack(
    model,
    checkpoint_path,
    preprocessing = dict(
        mean=[0.4812775254249573, 0.4674863815307617, 0.4093940854072571],
        std=[0.19709135591983795, 0.1933959424495697, 0.19051066040992737],
        axis=-3
    ),
    attack="fgsm",                  
    csv_path="./data/clean_data.csv",
    imdir="./images",
    outdir="./data",
    use_cuda=True,
    model_device=None,
    load_map_location=None,
    batch_size=32,
    verbose=True,
):
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    if model_device is None:
        device = torch.device("cuda:0" if (torch.cuda.is_available() and use_cuda) else "cpu")
    else:
        device = torch.device(model_device)

    if load_map_location is None:
        ckpt_map = "cpu" if device.type == "cpu" else device
    else:
        ckpt_map = load_map_location

    if verbose:
        print("Loading checkpoint:", checkpoint_path, "map_location:", ckpt_map)
    model = _load_checkpoint_into_model(model, checkpoint_path, map_location=ckpt_map, strict=True, verbose=verbose)

    model = model.to(device).eval()
    fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

    to_tensor = transforms.ToTensor()

    def load_image_tensor(rel_path, base_dir):
        p = Path(base_dir) / str(rel_path).lstrip("/")
        img = Image.open(p).convert("RGB")
        return to_tensor(img)

    if attack == "fgsm":
        atk = FGSM()
        epsilons = [0.03]
    elif attack == "bim":
        atk = LinfBasicIterativeAttack()
        epsilons = [0.03]
    elif attack == "pgd":
        atk = PGD()
        epsilons = [0.03]
    elif attack == "df":
        atk = L2DeepFoolAttack()
        epsilons = None
    elif attack == "cw":
        atk = L2CarliniWagnerAttack(steps=1000)
        epsilons = None
    else:
        raise ValueError("Unknown attack: " + str(attack))

    df = pd.read_csv(csv_path)

    df_correct = df[df["true_idx"] == df["pred_idx"]].copy()
    df_correct['original_index'] = df_correct.index
    total_candidates = len(df_correct)

    if verbose:
        print(f"CSV: {csv_path} rows={len(df)}; correctly-classified candidates={total_candidates}; epsilons={epsilons}")

    out_base = Path(outdir)
    out_base.mkdir(parents=True, exist_ok=True)

    attempted_count = 0
    success_count = 0
    saved_count = 0
    
    candidate_results = {}

    for i in tqdm(range(0, total_candidates, batch_size), desc="Processing Batches"):
        batch_df = df_correct.iloc[i:i+batch_size]
        
        images, labels, batch_info = [], [], []
        for _, row in batch_df.iterrows():
            try:
                img_tensor = load_image_tensor(row["rel_path"], imdir)
                images.append(img_tensor)
                labels.append(int(row["true_idx"]))
                batch_info.append(row)
            except Exception as e:
                if verbose:
                    print(f"Skipping image {row['rel_path']} due to loading error: {e}")
                candidate_results[row['original_index']] = {"success": False}
        
        if not images:
            continue
            
        images_t = torch.stack(images).to(device)
        labels_t = torch.tensor(labels, device=device)
        
        try:
            _, advs_t, success_t = atk(fmodel, images_t, criterion=foolbox.criteria.Misclassification(labels_t), epsilons=epsilons)
            attempted_count += len(images)
        except Exception as e:
            if verbose:
                print(f"Attack failed for a batch: {e}")
            for row in batch_info:
                candidate_results[row['original_index']] = {"success": False}
            continue

        for j, row in enumerate(batch_info):
            original_idx = row['original_index']
            
            if epsilons is not None:
                is_successful = bool(success_t[0][j].item())
                adv_tensor_for_item = advs_t[0][j]
            else: # For attacks like DeepFool where epsilons is None
                is_successful = bool(success_t[j].item())
                adv_tensor_for_item = advs_t[j]

            candidate_results[original_idx] = {"success": is_successful}
            
            if is_successful:
                success_count += 1
                adv_cpu = adv_tensor_for_item.cpu()
                
                try:
                    rel_clean = str(row["rel_path"]).lstrip("/")
                    out_rel = Path(rel_clean).with_suffix(".pt")
                    out_path = out_base / attack / out_rel
                    out_path.parent.mkdir(parents=True, exist_ok=True)
                    torch.save(adv_cpu, out_path)
                    saved_count += 1
                except Exception as e:
                    if verbose:
                        print(f"Failed to save adv tensor for {row['rel_path']}: {e}")

    df['success'] = df.index.map(lambda idx: candidate_results.get(idx, {}).get('success', False))
    
    meta_df = df[["rel_path", "true_idx", "pred_idx", "true_class", "pred_class", "success"]]
    meta_csv_path = out_base / f"metadata_{attack}.csv"
    meta_df.to_csv(meta_csv_path, index=False)
    if verbose:
        print(f"Wrote metadata CSV to: {meta_csv_path}")

    success_rate_over_candidates = success_count / total_candidates if total_candidates > 0 else 0.0
    success_rate_over_attempts = success_count / attempted_count if attempted_count > 0 else 0.0

    if verbose:
        print(f"Attempted attacks: {attempted_count}/{total_candidates} candidates.")
        print(f"Successes: {success_count}.")
        print(f"Saved adv files: {saved_count}.")
        print(f"Success rate (over candidates): {success_rate_over_candidates:.4f} ({success_count}/{total_candidates})")
        print(f"Success rate (over attempted):  {success_rate_over_attempts:.4f} ({success_count}/{attempted_count if attempted_count > 0 else 0})")

    return success_rate_over_candidates

In [4]:
import os
import py7zr

def compress_generated_images(foldername):
    src_dir = f'/kaggle/working/generated_images-test/{foldername}/'
    output_path = f'/kaggle/working/generated_images-test/{foldername}_stored.7z'

    if not os.path.exists(src_dir):
        raise FileNotFoundError(f"Source directory not found: {src_dir}")

    if os.path.exists(output_path):
        os.remove(output_path)

    with py7zr.SevenZipFile(
        output_path,
        mode='w',
        filters=[{'id': py7zr.FILTER_COPY}]
    ) as archive:
        archive.writeall(src_dir, arcname='.')

    print(f"Successfully archived : {output_path}")


In [5]:
from pathlib import Path

input_csv = '/kaggle/input/inatset/01-CleanModel/Evaluate/correct_in_all_models.csv'  
rows_per_file = 30          
output_dir = "split_csv"     

Path(output_dir).mkdir(exist_ok=True)

df = pd.read_csv(input_csv)

for i in range(0, len(df), rows_per_file):
    chunk = df.iloc[i:i + rows_per_file]
    chunk_file = Path(output_dir) / f"correct_in_all_models_{i//rows_per_file + 1}.csv"
    chunk.to_csv(chunk_file, index=False)
    print(f"Saved {chunk_file}")

print("Done! All chunks saved.")

Saved split_csv/correct_in_all_models_1.csv
Saved split_csv/correct_in_all_models_2.csv
Saved split_csv/correct_in_all_models_3.csv
Saved split_csv/correct_in_all_models_4.csv
Saved split_csv/correct_in_all_models_5.csv
Saved split_csv/correct_in_all_models_6.csv
Saved split_csv/correct_in_all_models_7.csv
Saved split_csv/correct_in_all_models_8.csv
Saved split_csv/correct_in_all_models_9.csv
Saved split_csv/correct_in_all_models_10.csv
Saved split_csv/correct_in_all_models_11.csv
Saved split_csv/correct_in_all_models_12.csv
Saved split_csv/correct_in_all_models_13.csv
Saved split_csv/correct_in_all_models_14.csv
Saved split_csv/correct_in_all_models_15.csv
Saved split_csv/correct_in_all_models_16.csv
Saved split_csv/correct_in_all_models_17.csv
Saved split_csv/correct_in_all_models_18.csv
Saved split_csv/correct_in_all_models_19.csv
Saved split_csv/correct_in_all_models_20.csv
Saved split_csv/correct_in_all_models_21.csv
Saved split_csv/correct_in_all_models_22.csv
Saved split_csv/cor

In [5]:
from torchvision.models import VGG16_BN_Weights
from torchvision import models, transforms

model = models.vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)
model.avgpool = nn.AdaptiveAvgPool2d((7,7))
model.classifier[6] = nn.Linear(4096, 7)

img_size = 512
checkpoint_path = f'/kaggle/input/inatset/01-CleanModel/Models/{img_size}x{img_size}/best_min_acc_vgg16_{img_size}x{img_size}_Model-2.pth'
imdir = f'/kaggle/input/inatset/01-CleanModel/Dataset/{img_size}x{img_size}'
outdir = f'./generated_images-test/{img_size}x{img_size}'

attack_method = 'cw'
for i in range (69,72):
    detect_csv_split = f'./split_csv/correct_in_all_models_{i}.csv'
    foldername = f'{outdir}-{i}'
    run_attack(
        model=model,
        checkpoint_path=checkpoint_path,                  
        attack="cw",                    
        csv_path=detect_csv_split,
        imdir=imdir,
        outdir=foldername,
        batch_size=2,
        verbose = True)
    compress_generated_images (f'512x512-{i}')

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:03<00:00, 181MB/s]  


Loading checkpoint: /kaggle/input/inatset/01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth map_location: cuda:0
Loaded checkpoint /kaggle/input/inatset/01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth (strict=True).
CSV: ./split_csv/correct_in_all_models_69.csv rows=30; correctly-classified candidates=30; epsilons=None


Processing Batches: 100%|██████████| 15/15 [46:08<00:00, 184.56s/it]


Wrote metadata CSV to: generated_images-test/512x512-69/metadata_cw.csv
Attempted attacks: 30/30 candidates.
Successes: 30.
Saved adv files: 30.
Success rate (over candidates): 1.0000 (30/30)
Success rate (over attempted):  1.0000 (30/30)
Successfully archived : /kaggle/working/generated_images-test/512x512-69_stored.7z
Loading checkpoint: /kaggle/input/inatset/01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth map_location: cuda:0
Loaded checkpoint /kaggle/input/inatset/01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth (strict=True).
CSV: ./split_csv/correct_in_all_models_70.csv rows=30; correctly-classified candidates=30; epsilons=None


Processing Batches: 100%|██████████| 15/15 [47:10<00:00, 188.70s/it]


Wrote metadata CSV to: generated_images-test/512x512-70/metadata_cw.csv
Attempted attacks: 30/30 candidates.
Successes: 30.
Saved adv files: 30.
Success rate (over candidates): 1.0000 (30/30)
Success rate (over attempted):  1.0000 (30/30)
Successfully archived : /kaggle/working/generated_images-test/512x512-70_stored.7z
Loading checkpoint: /kaggle/input/inatset/01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth map_location: cuda:0
Loaded checkpoint /kaggle/input/inatset/01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth (strict=True).
CSV: ./split_csv/correct_in_all_models_71.csv rows=30; correctly-classified candidates=30; epsilons=None


Processing Batches: 100%|██████████| 15/15 [41:32<00:00, 166.16s/it]


Wrote metadata CSV to: generated_images-test/512x512-71/metadata_cw.csv
Attempted attacks: 30/30 candidates.
Successes: 30.
Saved adv files: 30.
Success rate (over candidates): 1.0000 (30/30)
Success rate (over attempted):  1.0000 (30/30)
Successfully archived : /kaggle/working/generated_images-test/512x512-71_stored.7z
